!pip install google
!pip install google-cloud
!pip install google-cloud-language

In [1]:
import argparse
import pandas as pd
import os

from google.cloud import language_v1

In [2]:
'''
initial setting
'''
# service key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service_secret_key_nlpkosi01.json'


'''
신문 본문(data.body 중 하나, 여기서는 'article'로 표현)을 집어넣으면
해당 신문기사의 (긍정적인 정도 , 감정적인 정도) tuple이 return 됩니다.

ex, (0.5 , 5.5) 의 의미?
    -> 해당 신문기사가 0.5 만큼 긍정적이고, 그 감정의 강도는 5.5 라는 의미
'''
def analyze(article) -> tuple:
    client = language_v1.LanguageServiceClient()
    content = article
    
    document = language_v1.Document(content=content, type_=language_v1.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(request = {'document':document})
    
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    
    return score, magnitude
    
    
    
    
'''사용하지 않을 함수. (debug용)'''
def print_result(annotations):
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    
    
    for index, sentence in enumerate(annotations.sentences):
        sentence_sentiment = sentence.sentiment.score
        print("Sentence {} has a sentiment score of {}".format(index, sentence_sentiment))
    print("Overall sentiment: score of {} with magnitude of {}".format(score, magnitude))
    
    return 0

# sentiment analysis

In [30]:
datafile = pd.read_csv('신문기사모음(9.17)(utf8).csv')
data = datafile.body


data_dict = {}

for idx in range(len(data)):
    article = data[idx]
    
    # data processing & save data into dictionary
    data_dict[idx] = analyze(article)
    
result = pd.DataFrame(data_dict).transpose().reset_index()
result.columns = ['index','score', 'magnitude']
result_merged = pd.merge(datafile, result, how = 'left', on = 'index')
del result_merged['index']

# result
result_merged

{0: (0.5, 3.4000000953674316),
 1: (0.20000000298023224, 1.5),
 2: (0.20000000298023224, 0.20000000298023224),
 3: (0.0, 1.5),
 4: (-0.10000000149011612, 1.600000023841858),
 5: (0.4000000059604645, 1.7000000476837158),
 6: (0.4000000059604645, 12.5),
 7: (0.20000000298023224, 9.600000381469727),
 8: (0.10000000149011612, 12.600000381469727),
 9: (0.20000000298023224, 16.799999237060547),
 10: (0.0, 3.799999952316284),
 11: (0.10000000149011612, 4.699999809265137),
 12: (0.10000000149011612, 4.699999809265137),
 13: (0.20000000298023224, 3.5999999046325684),
 14: (0.20000000298023224, 10.600000381469727),
 15: (0.30000001192092896, 3.9000000953674316),
 16: (0.20000000298023224, 3.9000000953674316),
 17: (0.30000001192092896, 2.4000000953674316),
 18: (0.30000001192092896, 8.399999618530273),
 19: (0.20000000298023224, 3.700000047683716),
 20: (0.20000000298023224, 4.099999904632568),
 21: (0.20000000298023224, 3.700000047683716),
 22: (0.20000000298023224, 3.299999952316284),
 23: (0.

# 크롤링한 신문기사 재가공 (합쳐서 utf-8로)

In [29]:
os.listdir()

table = pd.DataFrame()

for file in os.listdir():
    if file[-7:] != '크롤링.csv':
        continue
    print(file)
    try:
        data = pd.read_csv(file, encoding = 'cp949', index_col = 0)
    except:
        continue
    if len(data) == 0:
        continue
    table = pd.concat([table, data])
    
table.to_csv('신문기사모음(9.17)(utf8).csv', encoding = 'utf-8')
    
    

경향 크롤링.csv
동아일보 크롤링.csv
디지털타임스 크롤링.csv
매일경제 크롤링.csv
전자신문 크롤링.csv
조선일보 크롤링.csv
중앙일보 크롤링.csv
한겨레 크롤링.csv
한국경제 크롤링.csv
